In [1]:
%%markdown
## Análise de dados

## Análise de dados


In [2]:
 # import libraries
from delta.tables import *
from pyspark.sql import SparkSession
# from pyspark import SparkConf
# from pyspark.sql import *
import pyspark
from delta import *

# Adicionado um outro caminho para o warehouse
builder = SparkSession.builder.appName("Analytic") \
.config("spark.sql.warehouse.dir", '/home/fovr/PROJETOS/olist-ml-models/src') \
.config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \
.config("spark.jars", "/home/fovr/PROJETOS/olist-ml-models/jdbc/sqlite-jdbc-3.34.0.jar")  \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.enableHiveSupport() 

spark = configure_spark_with_delta_pip(builder).getOrCreate()



:: loading settings :: url = jar:file:/opt/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fovr/.ivy2/cache
The jars for the packages stored in: /home/fovr/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5476b38f-2d68-4ca1-8755-2b68f1470d75;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 267ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

23/04/10 10:54:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
%%markdown
### Silver
silver.tbAvaliacao \
silver.tbCliente \
silver.tbGeolocaliza \
silver.tbItemPedido \
silver.tbPagamentoPedido \
silver.tbPedido \
silver.tbProduto \
silver.tbVendedor 

### Silver
silver.tbAvaliacao \
silver.tbCliente \
silver.tbGeolocaliza \
silver.tbItemPedido \
silver.tbPagamentoPedido \
silver.tbPedido \
silver.tbProduto \
silver.tbVendedor 


In [39]:
%%markdown
# Pagamentos

### Issues
* formas de pagamentos
* Quantidade média de parcelas

# Pagamentos

### Issues
* formas de pagamentos
* Quantidade média de parcelas


In [8]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|   silver|
+---------+



In [9]:
spark.sql(""" SELECT * FROM silver.tbPedido""").show(5)

+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|            idPedido|           idCliente|descSituacao|  dtPedido|dtAprovado|   dtEnvio|dtEntregue|dtEstimativaEntrega|
+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|15324bb6dca4436f4...|191ba5e82034dafb0...|   delivered|2017-02-14|2017-02-14|2017-02-15|2017-02-20|         2017-03-15|
|530e59f0f148ec002...|af350704fe4d751c6...|   delivered|2017-02-14|2017-02-14|2017-02-15|2017-02-17|         2017-03-13|
|35dc5c3a87f9e9cf9...|be59c56551fe3cef6...| unavailable|2017-02-14|2017-02-14|      null|      null|         2017-03-15|
|e706d614326b76f48...|6fd87c3e71f1826c3...|   delivered|2017-02-14|2017-02-14|2017-02-17|2017-03-03|         2017-03-23|
|d2d05762a4554c9d9...|5b43ef89a22571e79...|   delivered|2017-02-14|2017-02-15|2017-02-24|2017-03-08|         2017-03-21|
+--------------------+----------

In [10]:
# Filtro para pegar apenas os ultimos 6 meses

spark.sql("""

SELECT * 
FROM silver.tbPedido
WHERE dtPedido < '2018-01-01' AND dtPedido >= add_months('2018-01-01', -6)

""").show()

+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|            idPedido|           idCliente|descSituacao|  dtPedido|dtAprovado|   dtEnvio|dtEntregue|dtEstimativaEntrega|
+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|d445a6e277eaa6290...|c2cd065252fa7ad04...|   delivered|2017-08-03|2017-08-03|2017-08-07|2017-08-14|         2017-09-06|
|966d505c98ea667c1...|26b32bdec947c658c...|   delivered|2017-08-03|2017-08-05|2017-08-16|2017-08-25|         2017-09-01|
|b682fc4ea655d9de9...|99c336ab501ab1c1e...|   delivered|2017-08-03|2017-08-05|2017-08-09|2017-08-28|         2017-08-23|
|52324f390c9414a6c...|5a9dae6565b54b1a3...|   delivered|2017-08-03|2017-08-04|2017-08-04|2017-08-11|         2017-09-06|
|8f2c9e626786d2b53...|98a505463599ce446...|   delivered|2017-08-03|2017-08-05|2017-08-07|2017-08-14|         2017-08-29|
|b8cd3107b41b1232e...|e7958b1dac

In [11]:

# Base distintas de pedidos ao longo de seis meses
spark.sql("""

SELECT DISTINCT 

A.idPedido,
B.idVendedor

FROM silver.tbPedido AS A
LEFT JOIN silver.tbItemPedido AS B ON A.idPedido = B.idPedido
 WHERE A.dtPedido < '2018-01-01' 
        AND A.dtPedido >= add_months('2018-01-01', -6) 
        AND idVendedor IS NOT NULl

""").show(truncate=False)

+--------------------------------+--------------------------------+
|idPedido                        |idVendedor                      |
+--------------------------------+--------------------------------+
|00f1c7c1542f5f86c3091749d9e939d8|cc419e0650a3c5ba77189a1882b7556a|
|049999b745ab7d4bc3ddf00181c39d56|ea8482cd71df3c1969d7b9473ff13abc|
|055a2893cb600bebc05a16d06e3de36f|6426d21aca402a131fc0a5d0960a3c90|
|08c79f0b250cc3b309a69dcbd0debdc7|c31eff8334d6b3047ed34bebd4d62c36|
|097c6926697b318e2e49c0d92843211a|cbd996ad3c1b7dc71fd0e5f5df9087e2|
|1051983b1aca1ef8f4f6827378c12c8a|7e93a43ef30c4f03f38b393420bc753a|
|11f1347027d269d264f62d3631ef5e42|4a3ca9315b744ce9f8e9374361493884|
|166a835dcb0ffdb5d917db2a93358502|0c7533c71df861ec58ad7ff999ed0e8d|
|17eea1bb78f458449eacc5933303cbf8|669ae81880e08f269a64487cfb287169|
|1a96dbd0041decf9c11c051b4f13d676|91f5779a8731baca4809fdeade81803f|
|1b4302389eccf1e9df072501f38b1e9a|04e38a233f726ef161074ab1c9b0399a|
|1b99f13e8fda59c0a43efdbe69e16c8c|4e922959ae960d

In [19]:
# Estatisticas de cada vendedor que fizeram pedidos em um período de 6 meses.

df = spark.sql("""

WITH 

tb_pedido AS (

SELECT DISTINCT 

A.idPedido,
B.idVendedor

FROM silver.tbPedido AS A
LEFT JOIN silver.tbItemPedido AS B ON A.idPedido = B.idPedido
 WHERE A.dtPedido < '2018-01-01' 
        AND A.dtPedido >= add_months('2018-01-01', -6) 
        AND idVendedor IS NOT NULl

), 


tb1 AS (

    SELECT 
   C.idVendedor,
    B.*
        FROM tb_pedido AS C
        LEFT JOIN silver.tbPagamentoPedido as B ON C.idPedido = B.idPedido ),
       
      

    tb2 AS (
        
        SELECT idVendedor,
            descTipoPagamento,
            COUNT(DISTINCT idPedido) AS qtPedidoMeioPagamento,
            SUM(vlPagamento) AS vlPedidoPagamento

        FROM tb1

        GROUP BY idVendedor, descTipoPagamento
        ORDER BY idVendedor, descTipoPagamento ),
        
   tbsummary AS ( SELECT  idVendedor 
   
             , SUM(CASE WHEN descTipoPagamento='boleto' then qtPedidoMeioPagamento else 0 end) as qt_boleto_pedido
             , SUM(CASE WHEN descTipoPagamento='credit_card' then qtPedidoMeioPagamento else 0 end) as qt_credit_card_pedido
             , SUM(CASE WHEN descTipoPagamento='voucher' then qtPedidoMeioPagamento else 0 end) as qt_voucher_pedido
             , SUM(CASE WHEN descTipoPagamento='debit_card' then qtPedidoMeioPagamento else 0 end) as qt_debit_card_pedido
    
             , SUM(CASE WHEN descTipoPagamento = 'boleto' THEN vlPedidoPagamento ELSE 0 END) AS vl_boleto_pedido           
             , SUM(CASE WHEN descTipoPagamento = 'credit_card' THEN vlPedidoPagamento ELSE 0 END) AS vl_credit_card_pedido      
             , SUM(CASE WHEN descTipoPagamento = 'voucher' THEN vlPedidoPagamento ELSE 0 END) AS vl_voucher_pedido           
             , SUM(CASE WHEN descTipoPagamento = 'debit_card' THEN vlPedidoPagamento ELSE 0 END) AS vl_debit_card_pedido
            
            , SUM(CASE WHEN descTipoPagamento = 'boleto' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_boleto_pedido           
            , SUM(CASE WHEN descTipoPagamento = 'credit_card' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_credit_card_pedido      
            , SUM(CASE WHEN descTipoPagamento = 'voucher' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_voucher_pedido           
            , SUM(CASE WHEN descTipoPagamento = 'debit_card' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_debit_card_pedido
            
            FROM   tb2
    GROUP BY idVendedor  ),
    
    
    tb_cartao AS (
    
            
            SELECT idVendedor,
                 AVG(nrParcelas) AS avgQtdeParcelas, 
                 PERCENTILE(nrParcelas, 0.5) AS medianQtdeParcelas,
                 MAX(nrParcelas) AS maxQtdeParcelas,
                 MIN(nrParcelas) AS minQtdeParcelas

          FROM tb1

          WHERE descTipoPagamento = 'credit_card'

          GROUP BY idVendedor

    )
    
            SELECT 
               '2018-01-01' AS dtReference,
               t1.*,
               t2.avgQtdeParcelas AS mediaQtParcelas,
               t2.medianQtdeParcelas,
               t2.maxQtdeParcelas,
               t2.minQtdeParcelas

        FROM tbsummary as t1

        LEFT JOIN tb_cartao as t2
        ON t1.idVendedor = t2.idVendedor
    

""")
df.createOrReplaceTempView('VwSummary')
df.show(truncate=False, vertical= True)



-RECORD 0-----------------------------------------------------
 dtReference               | 2018-01-01                       
 idVendedor                | 062ce95fa2ad4dfaedfc79260130565f 
 qt_boleto_pedido          | 2                                
 qt_credit_card_pedido     | 10                               
 qt_voucher_pedido         | 0                                
 qt_debit_card_pedido      | 0                                
 vl_boleto_pedido          | 114.20000076293945               
 vl_credit_card_pedido     | 1924.2600212097168               
 vl_voucher_pedido         | 0.0                              
 vl_debit_card_pedido      | 0.0                              
 pct_qt_boleto_pedido      | 0.16666666666666666              
 pct_qt_credit_card_pedido | 0.8333333333333334               
 pct_qt_voucher_pedido     | 0.0                              
 pct_qt_debit_card_pedido  | 0.0                              
 mediaQtParcelas           | 4.4                       

In [40]:
%%markdown
# Clientes

### Issues

# Clientes

### Issues


In [24]:
df_clientes = spark.sql("""

WITH tb_join AS (

  SELECT DISTINCT
         t1.idPedido,
         t1.idCliente,
         t2.idVendedor,
         t3.descUF

  FROM silver.tbPedido AS  t1

  LEFT JOIN silver.tbItemPedido as t2
  ON t1.idPedido = t2.idPedido

  LEFT JOIN silver.tbCliente as t3
  ON t1.idCliente = t3.idCliente

  WHERE dtPedido < '2018-01-01'
  AND dtPedido >= add_months('2018-01-01', -6)
  AND idVendedor IS NOT NULL

),

tb_group AS (

  SELECT
    idVendedor,

    count(distinct descUF) as qtdUFsPedidos,

    count(distinct case when descUF = 'AC' then idPedido end) / count(distinct idPedido) as pctPedidoAC,
    count(distinct case when descUF = 'AL' then idPedido end) / count(distinct idPedido) as pctPedidoAL,
    count(distinct case when descUF = 'AM' then idPedido end) / count(distinct idPedido) as pctPedidoAM,
    count(distinct case when descUF = 'AP' then idPedido end) / count(distinct idPedido) as pctPedidoAP,
    count(distinct case when descUF = 'BA' then idPedido end) / count(distinct idPedido) as pctPedidoBA,
    count(distinct case when descUF = 'CE' then idPedido end) / count(distinct idPedido) as pctPedidoCE,
    count(distinct case when descUF = 'DF' then idPedido end) / count(distinct idPedido) as pctPedidoDF,
    count(distinct case when descUF = 'ES' then idPedido end) / count(distinct idPedido) as pctPedidoES,
    count(distinct case when descUF = 'GO' then idPedido end) / count(distinct idPedido) as pctPedidoGO,
    count(distinct case when descUF = 'MA' then idPedido end) / count(distinct idPedido) as pctPedidoMA,
    count(distinct case when descUF = 'MG' then idPedido end) / count(distinct idPedido) as pctPedidoMG,
    count(distinct case when descUF = 'MS' then idPedido end) / count(distinct idPedido) as pctPedidoMS,
    count(distinct case when descUF = 'MT' then idPedido end) / count(distinct idPedido) as pctPedidoMT,
    count(distinct case when descUF = 'PA' then idPedido end) / count(distinct idPedido) as pctPedidoPA,
    count(distinct case when descUF = 'PB' then idPedido end) / count(distinct idPedido) as pctPedidoPB,
    count(distinct case when descUF = 'PE' then idPedido end) / count(distinct idPedido) as pctPedidoPE,
    count(distinct case when descUF = 'PI' then idPedido end) / count(distinct idPedido) as pctPedidoPI,
    count(distinct case when descUF = 'PR' then idPedido end) / count(distinct idPedido) as pctPedidoPR,
    count(distinct case when descUF = 'RJ' then idPedido end) / count(distinct idPedido) as pctPedidoRJ,
    count(distinct case when descUF = 'RN' then idPedido end) / count(distinct idPedido) as pctPedidoRN,
    count(distinct case when descUF = 'RO' then idPedido end) / count(distinct idPedido) as pctPedidoRO,
    count(distinct case when descUF = 'RR' then idPedido end) / count(distinct idPedido) as pctPedidoRR,
    count(distinct case when descUF = 'RS' then idPedido end) / count(distinct idPedido) as pctPedidoRS,
    count(distinct case when descUF = 'SC' then idPedido end) / count(distinct idPedido) as pctPedidoSC,
    count(distinct case when descUF = 'SE' then idPedido end) / count(distinct idPedido) as pctPedidoSE,
    count(distinct case when descUF = 'SP' then idPedido end) / count(distinct idPedido) as pctPedidoSP,
    count(distinct case when descUF = 'TO' then idPedido end) / count(distinct idPedido) as pctPedidoTO

  FROM tb_join

  GROUP BY idVendedor

)

SELECT 
    '2018-01-01' AS dtReference,
    *

FROM tb_group

""")

df_clientes.show(5, vertical=True)

-RECORD 0-----------------------------
 dtReference   | 2018-01-01           
 idVendedor    | 0ea22c1cfbdc755f8... 
 qtdUFsPedidos | 13                   
 pctPedidoAC   | 0.0                  
 pctPedidoAL   | 0.0                  
 pctPedidoAM   | 0.0                  
 pctPedidoAP   | 0.0                  
 pctPedidoBA   | 0.014285714285714285 
 pctPedidoCE   | 0.0                  
 pctPedidoDF   | 0.05714285714285714  
 pctPedidoES   | 0.0                  
 pctPedidoGO   | 0.02857142857142857  
 pctPedidoMA   | 0.0                  
 pctPedidoMG   | 0.3                  
 pctPedidoMS   | 0.0                  
 pctPedidoMT   | 0.0                  
 pctPedidoPA   | 0.014285714285714285 
 pctPedidoPB   | 0.014285714285714285 
 pctPedidoPE   | 0.014285714285714285 
 pctPedidoPI   | 0.0                  
 pctPedidoPR   | 0.014285714285714285 
 pctPedidoRJ   | 0.1                  
 pctPedidoRN   | 0.0                  
 pctPedidoRO   | 0.0                  
 pctPedidoRR   | 0.0     

In [41]:
%%markdown
# Produtos

### Issues

# Produtos

### Issues


In [29]:
df_prod = spark.sql("""

WITH tb_join AS (

  SELECT DISTINCT
         t2.idVendedor,
         t3.*

  FROM silver.tbpedido AS t1

  LEFT JOIN silver.tbItemPedido AS t2
  ON t1.idPedido = t2.idPedido

  LEFT JOIN silver.tbProduto as t3
  ON t2.idProduto = t3.idProduto

  WHERE t1.dtPedido < '2018-01-01'
  AND t1.dtPedido >= add_months('2018-01-01', -6)
  AND t2.idVendedor IS NOT NULL

),

tb_summary as (

  SELECT idVendedor,
         avg(coalesce(nrFotos,0)) as avgFotos,
         avg(vlComprimentoCm * vlAlturaCm * vlLarguraCm) as avgVolumeProduto,
         percentile(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0.5) as medianVolumeProduto,
         min(vlComprimentoCm * vlAlturaCm * vlLarguraCm) as minVolumeProduto,
         max(vlComprimentoCm * vlAlturaCm * vlLarguraCm) as maxVolumeProduto,
         
        count(distinct case when descCategoria = 'cama_mesa_banho' then idProduto end) / count(distinct idProduto) as pctCategoriacama_mesa_banho,
        count(distinct case when descCategoria = 'beleza_saude' then idProduto end) / count(distinct idProduto) as pctCategoriabeleza_saude,
        count(distinct case when descCategoria = 'esporte_lazer' then idProduto end) / count(distinct idProduto) as pctCategoriaesporte_lazer,
        count(distinct case when descCategoria = 'informatica_acessorios' then idProduto end) / count(distinct idProduto) as pctCategoriainformatica_acessorios,
        count(distinct case when descCategoria = 'moveis_decoracao' then idProduto end) / count(distinct idProduto) as pctCategoriamoveis_decoracao,
        count(distinct case when descCategoria = 'utilidades_domesticas' then idProduto end) / count(distinct idProduto) as pctCategoriautilidades_domesticas,
        count(distinct case when descCategoria = 'relogios_presentes' then idProduto end) / count(distinct idProduto) as pctCategoriarelogios_presentes,
        count(distinct case when descCategoria = 'telefonia' then idProduto end) / count(distinct idProduto) as pctCategoriatelefonia,
        count(distinct case when descCategoria = 'automotivo' then idProduto end) / count(distinct idProduto) as pctCategoriaautomotivo,
        count(distinct case when descCategoria = 'brinquedos' then idProduto end) / count(distinct idProduto) as pctCategoriabrinquedos,
        count(distinct case when descCategoria = 'cool_stuff' then idProduto end) / count(distinct idProduto) as pctCategoriacool_stuff,
        count(distinct case when descCategoria = 'ferramentas_jardim' then idProduto end) / count(distinct idProduto) as pctCategoriaferramentas_jardim,
        count(distinct case when descCategoria = 'perfumaria' then idProduto end) / count(distinct idProduto) as pctCategoriaperfumaria,
        count(distinct case when descCategoria = 'bebes' then idProduto end) / count(distinct idProduto) as pctCategoriabebes,
        count(distinct case when descCategoria = 'eletronicos' then idProduto end) / count(distinct idProduto) as pctCategoriaeletronicos

  FROM tb_join

  GROUP BY idVendedor

)

SELECT '2018-01-01' AS dtReference,
       *

FROM tb_summary

""")

df_prod.show(5, vertical=True)

-RECORD 0--------------------------------------------------
 dtReference                        | 2018-01-01           
 idVendedor                         | 062ce95fa2ad4dfae... 
 avgFotos                           | 1.8                  
 avgVolumeProduto                   | 4763.4               
 medianVolumeProduto                | 3705.0               
 minVolumeProduto                   | 1575.0               
 maxVolumeProduto                   | 9207.0               
 pctCategoriacama_mesa_banho        | 0.2                  
 pctCategoriabeleza_saude           | 0.0                  
 pctCategoriaesporte_lazer          | 0.0                  
 pctCategoriainformatica_acessorios | 0.0                  
 pctCategoriamoveis_decoracao       | 0.6                  
 pctCategoriautilidades_domesticas  | 0.0                  
 pctCategoriarelogios_presentes     | 0.0                  
 pctCategoriatelefonia              | 0.0                  
 pctCategoriaautomotivo             | 0.

In [42]:
%%markdown

# Vendas

### Issues



# Vendas

### Issues


In [33]:
df_vendas = spark.sql("""

WITH tb_pedido_item AS (

  SELECT t2.*,
         t1.dtPedido

  FROM silver.tbPedido AS t1

  LEFT JOIN silver.tbItemPedido AS t2
  ON t1.idPedido = t2.idPedido

  WHERE t1.dtPedido < '2018-01-01'
  AND t1.dtPedido >= add_months('2018-01-01', -6)
  AND t2.idVendedor IS NOT NULL

),

tb_summary AS (

  SELECT 
        idVendedor,
        count(distinct idPedido) AS qtdPedidos,
        count(distinct date(dtPedido)) AS qtdDias,
        count(idProduto) AS qtItens,
        datediff('2018-01-01', max(dtPedido)) AS qtdRecencia,
        sum(vlPreco) / count(distinct idPedido) as avgTicket,
        avg(vlPreco) AS avgValorProduto,
        max(vlPreco) AS maxValorProduto,
        min(vlPreco) AS minValorProduto,
        count(idProduto) / count(distinct idPedido) as avgProdutoPedido

  FROM tb_pedido_item
  
  GROUP BY idVendedor

),

tb_pedido_summary AS (

  SELECT idVendedor,
         idPedido,
         sum(vlPreco) as vlPreco

  FROM tb_pedido_item

  GROUP BY idVendedor, idPedido

),

tb_min_max AS (

  SELECT idVendedor,
         min(vlPreco) AS minVlPedido,
         max(vlPreco) AS maxVlPedido

  FROM tb_pedido_summary

  GROUP BY idVendedor

),

tb_life AS (

  SELECT t2.idVendedor,
         sum(vlPreco) AS LTV,
         max(datediff('2018-01-01', dtPedido)) AS qtdeDiasBase

  FROM silver.tbPedido AS t1

  LEFT JOIN silver.tbItemPedido AS t2
  ON t1.idPedido = t2.idPedido

  WHERE t1.dtPedido < '2018-01-01'
  AND t2.idVendedor IS NOT NULL

  GROUP BY t2.idVendedor
),

tb_dtpedido AS (

  SELECT distinct idVendedor,
         date(dtPedido) as dtPedido
  FROM tb_pedido_item

  ORDER BY 1,2

),

tb_lag (

  SELECT *,
         LAG(dtPedido) OVER (PARTITION BY idVendedor ORDER BY dtPedido) AS lag1
  FROM tb_dtpedido

),

tb_intervalo AS (

  SELECT idVendedor,
         avg(datediff(dtPedido, lag1)) as avgIntervaloVendas
  FROM tb_lag
  GROUP BY idVendedor

)

SELECT 
       '2018-01-01' AS dtReference,
       t1.*,
       t2.minVlPedido,
       t2.maxVlPedido,
       t3.LTV,
       t3.qtdeDiasBase,
       t4.avgIntervaloVendas

FROM tb_summary as t1

LEFT JOIN tb_min_max AS t2
ON t1.idVendedor = t2.idVendedor

LEFT JOIN tb_life AS t3
on t1.idVendedor = t3.idVendedor

LEFT JOIN tb_intervalo AS t4
ON t1.idVendedor = t4.idVendedor

""")

df_vendas.show(5, vertical = True)

-RECORD 0----------------------------------
 dtReference        | 2018-01-01           
 idVendedor         | 062ce95fa2ad4dfae... 
 qtdPedidos         | 12                   
 qtdDias            | 11                   
 qtItens            | 12                   
 qtdRecencia        | 113                  
 avgTicket          | 133.3999983469645    
 avgValorProduto    | 133.3999983469645    
 maxValorProduto    | 369.9                
 minValorProduto    | 39.9                 
 avgProdutoPedido   | 1.0                  
 minVlPedido        | 39.900001525878906   
 maxVlPedido        | 369.8999938964844    
 LTV                | 7520.999885559082    
 qtdeDiasBase       | 340                  
 avgIntervaloVendas | 6.9                  
-RECORD 1----------------------------------
 dtReference        | 2018-01-01           
 idVendedor         | 0ea22c1cfbdc755f8... 
 qtdPedidos         | 70                   
 qtdDias            | 63                   
 qtItens            | 79        

In [48]:
%%markdown

# Entrega

### Issues


# Entrega

### Issues


In [38]:
df_entrega = spark.sql("""

WITH tb_pedido AS (

SELECT t1.idPedido,
       t2.idVendedor,
       t1.descSituacao,
       t1.dtPedido,
       t1.dtAprovado,
       t1.dtEntregue,
       t1.dtEstimativaEntrega,
       sum(vlFrete) as totalFrente       

FROM silver.tbPedido AS t1

LEFT JOIN silver.tbItemPedido as t2
ON t1.idPedido = t2.idPedido

WHERE dtPedido < '2018-01-01'
AND dtPedido >= add_months('2018-01-01', -6)
AND idVendedor IS NOT NULL

GROUP BY t1.idPedido,
         t2.idVendedor,
         t1.descSituacao,
         t1.dtPedido,
         t1.dtAprovado,
         t1.dtEntregue,
         t1.dtEstimativaEntrega
)

SELECT
    '2018-01-01' AS dtReference,
    idVendedor,
    COUNT(DISTINCT CASE WHEN date(coalesce(dtEntregue, '2018-01-01')) > date(dtEstimativaEntrega) THEN idPedido END) / COUNT(DISTINCT CASE WHEN descSituacao = 'delivered' THEN idPedido END) AS pctPedidoAtraso,
    count(distinct case when descSituacao = 'canceled' then idPedido end) / count(distinct idPedido) AS pctPedidoCancelado,
    avg(totalFrente) as avgFrete,
    percentile(totalFrente, 0.5) as medianFrete,
    max(totalFrente) as maxFrete,
    min(totalFrente) as minFrete,
    avg(datediff(coalesce(dtEntregue, '2018-01-01'), dtAprovado)) AS qtdDiasAprovadoEntrega,
    avg(datediff(coalesce(dtEntregue, '2018-01-01'), dtPedido)) AS qtdDiasPedidoEntrega,
    avg(datediff(dtEstimativaEntrega, coalesce(dtEntregue, '2018-01-01'))) AS qtdeDiasEntregaPromessa
       
FROM tb_pedido

GROUP BY 1, 2

""")

df_entrega.show(5, vertical=True)

-RECORD 0---------------------------------------
 dtReference             | 2018-01-01           
 idVendedor              | 062ce95fa2ad4dfae... 
 pctPedidoAtraso         | 0.2727272727272727   
 pctPedidoCancelado      | 0.0                  
 avgFrete                | 19.795000046491623   
 medianFrete             | 19.244999885559082   
 maxFrete                | 42.380001068115234   
 minFrete                | 1.100000023841858    
 qtdDiasAprovadoEntrega  | 26.75                
 qtdDiasPedidoEntrega    | 27.0                 
 qtdeDiasEntregaPromessa | -2.0                 
-RECORD 1---------------------------------------
 dtReference             | 2018-01-01           
 idVendedor              | 0ea22c1cfbdc755f8... 
 pctPedidoAtraso         | 0.014285714285714285 
 pctPedidoCancelado      | 0.0                  
 avgFrete                | 16.966000175476076   
 medianFrete             | 14.455000400543213   
 maxFrete                | 56.70000076293945    
 minFrete           

In [47]:
%%markdown

# Avaliação

### Issues

* Avaliação média em NPS
* Nota média da avaliação
* Qtd de avaliações
* Qtd de avaliações negativas


# Avaliação

### Issues

* Avaliação média em NPS
* Nota média da avaliação
* Qtd de avaliações
* Qtd de avaliações negativas


In [46]:
df_avaliacao = spark.sql("""

WITH tb_pedido AS (

  SELECT DISTINCT
         t1.idPedido,
         t2.idVendedor

  FROM silver.tbPedido as t1

  LEFT JOIN silver.tbItemPedido as t2
  ON t1.idPedido = t2.idPedido

  WHERE t1.dtPedido < '2018-01-01'
  AND t1.dtPedido >= add_months('2018-01-01', -6)
  AND idVendedor IS NOT NULL

),

tb_join AS (

  SELECT t1.*,
         t2.vlNota       

  FROM tb_pedido AS t1

  LEFT JOIN silver.tbAvaliacao AS t2
  ON t1.idPedido = t2.idPedido

),

tb_summary AS (

  SELECT 
      idVendedor,
      avg(vlNota) as avgNota,
      percentile(vlNota, 0.5) as medianNota,
      min(vlNota) as minNota,
      max(vlNota) as maxNota,
      count(vlNota) / count(idPedido) as pctAvaliacao

  FROM tb_join

  GROUP BY idVendedor

)

SELECT '2018-01-01' AS dtReference,
       *
FROM tb_summary

""")

df_avaliacao.show(5, vertical=True)

-RECORD 0----------------------------
 dtReference  | 2018-01-01           
 idVendedor   | 0015a82c2db000af6... 
 avgNota      | 3.6666666666666665   
 medianNota   | 5.0                  
 minNota      | 1.0                  
 maxNota      | 5.0                  
 pctAvaliacao | 1.0                  
-RECORD 1----------------------------
 dtReference  | 2018-01-01           
 idVendedor   | 001cca7ae9ae17fb1... 
 avgNota      | 4.135416666666667    
 medianNota   | 5.0                  
 minNota      | 1.0                  
 maxNota      | 5.0                  
 pctAvaliacao | 0.9795918367346939   
-RECORD 2----------------------------
 dtReference  | 2018-01-01           
 idVendedor   | 002100f778ceb8431... 
 avgNota      | 3.948717948717949    
 medianNota   | 5.0                  
 minNota      | 1.0                  
 maxNota      | 5.0                  
 pctAvaliacao | 1.0                  
-RECORD 3----------------------------
 dtReference  | 2018-01-01           
 idVendedor 